## 5.1 Enrichment

In [ ]:
library(clusterProfiler)
library(org.Hs.eg.db) 
library(AnnotationDbi)
library(DOSE)
library(circlize)

filtered_markers <- sub.adi.sc.markers %>% filter(!grepl("^ENSSSCG", gene))
New.markers <- filtered_markers %>%
  group_by(cluster) %>%
  top_n(25, wt = avg_log2FC) %>%
  ungroup()

markers_by_cluster <- split(New.markers$gene, New.markers$cluster)

go_results <- list()

for (cluster in names(markers_by_cluster)) {
    genes <- markers_by_cluster[[cluster]]
    
    gene_entrez <- bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Hs.eg.db)
    
    go_results[[cluster]] <- enrichGO(gene = gene_entrez$ENTREZID,
                                      OrgDb = org.Hs.eg.db,
                                      ont = "ALL", # 生物过程
                                      pAdjustMethod = "BH",
                                      pvalueCutoff = 0.9,
                                      qvalueCutoff = 0.9,
                                      readable = TRUE)
}

new_go_results <- list()
for (cluster in names(go_results)) {
    go_result <- go_results[[cluster]]@result
    gene_ratio_split <- strsplit(as.character(go_result$GeneRatio), "/")
    gene_ratio_numeric <- do.call(rbind, gene_ratio_split)
    gene_ratio_numeric <- apply(gene_ratio_numeric, 2, as.numeric)
    filter_indices <- gene_ratio_numeric[, 1] > 1
    filtered_result <- go_result[filter_indices, ]
    new_go_results[[cluster]] <- filtered_result
}

library(circlize)
cluster <- "2"
go_result <- new_go_results[[cluster]]

go_result <- go_result[order(go_result$GeneRatio, decreasing = TRUE), ]
go_result <- head(go_result, 8)

data <- do.call(rbind, lapply(seq(nrow(go_result)), function(i) {
    genes <- unlist(strsplit(go_result$geneID[i], "/"))
    data.frame(from = genes, to = rep(go_result$Description[i], length(genes)))
}))

num_sectors <- length(unique(c(data$from, data$to)))

circos.clear()
circos.par(gap.degree = rep(1, num_sectors))

chordDiagram(
  data,
  transparency = 0.5,
  link.border = "black",
  grid.border = "black",
  annotationTrack = "grid",
  annotationTrackHeight = c(0.15, 0.15)  # 增加外层圆圈的厚度
)

title(paste("Cluster", cluster))

## 5.2 Comparative analysis 

In [ ]:
library(Seurat)
library(ggplot2)
library(dplyr)
library(tibble)

# Figure 5K
perform_diff_exp <- function(seurat_obj, celltype, tissue, p_val_adj_threshold = 0.05, log2fc_threshold = 1) {
  active.ident <- as.character(Idents(seurat_obj))
  meta_data <- seurat_obj@meta.data
  meta_data$active.ident <- active.ident
  specific_cells <- rownames(meta_data[meta_data$orig.ident == tissue & meta_data$active.ident == celltype, ])
  all_cells <- rownames(meta_data[meta_data$active.ident == celltype, ])
  meta_data$group <- NA
  meta_data$group[rownames(meta_data) %in% specific_cells] <- paste(tissue, celltype, sep = "_")
  meta_data$group[rownames(meta_data) %in% all_cells & is.na(meta_data$group)] <- paste("All", celltype, sep = "_")
  meta_data$group <- factor(meta_data$group, levels = c(paste(tissue, celltype, sep = "_"), paste("All", celltype, sep = "_")))
  seurat_obj@meta.data <- meta_data
  group_cells <- rownames(meta_data[!is.na(meta_data$group), ])
  seurat_obj_subset <- subset(seurat_obj, cells = group_cells)
  
  Idents(seurat_obj_subset) <- seurat_obj_subset$group
  diff_exp <- FindMarkers(seurat_obj_subset, ident.1 = paste(tissue, celltype, sep = "_"), ident.2 = paste("All", celltype, sep = "_"))
  
  diff_exp <- diff_exp %>%
    rownames_to_column(var = "gene") %>%
    mutate(Significance = ifelse(p_val_adj < p_val_adj_threshold & abs(avg_log2FC) > log2fc_threshold, "Significant", "Not Significant"))
  return(diff_exp)
}


diff_exp_result <- perform_diff_exp(sub.adi.sc, "MR-FAPs", "Sol", p_val_adj_threshold = 0.05, log2fc_threshold = 1)
print(head(diff_exp_result))

## 5.3 Comparative analysis 

In [ ]:
# Figure 5L
library(Seurat)
library(ggplot2)
library(cowplot)
library(dplyr)
library(ggrepel)
theme_set(theme_cowplot())

plot_scatter_with_top_genes <- function(seurat_obj, cell_group, tissue_1, tissue_2, threshold = 1e5, top_n = 10) {
    active.ident <- as.character(Idents(seurat_obj))
    meta_data <- seurat_obj@meta.data
    meta_data$active.ident <- active.ident
    
    if (tissue_2 == "ALL") {
        tissue_1_cells <- rownames(meta_data[meta_data$orig.ident == tissue_1 & meta_data$active.ident == cell_group, ])
        
        all_cells <- rownames(meta_data[meta_data$active.ident == cell_group, ])
        
        meta_data$group <- NA
        meta_data$group[rownames(meta_data) %in% tissue_1_cells] <- paste0(tissue_1, "_", cell_group)
        meta_data$group[rownames(meta_data) %in% all_cells & is.na(meta_data$group)] <- paste0("ALL_", cell_group)
    } else {
        tissue_1_cells <- rownames(meta_data[meta_data$orig.ident == tissue_1 & meta_data$active.ident == cell_group, ])
        tissue_2_cells <- rownames(meta_data[meta_data$orig.ident == tissue_2 & meta_data$active.ident == cell_group, ])
        
        meta_data$group <- NA
        meta_data$group[rownames(meta_data) %in% tissue_1_cells] <- paste0(tissue_1, "_", cell_group)
        meta_data$group[rownames(meta_data) %in% tissue_2_cells] <- paste0(tissue_2, "_", cell_group)
    }
    
    meta_data$group <- factor(meta_data$group, levels = unique(meta_data$group))
    seurat_obj@meta.data <- meta_data

    print(table(meta_data$group))
    
    aggregate_data <- AggregateExpression(seurat_obj, group.by = "group", assays = "RNA", slot = "data")
    
    data <- as.data.frame(aggregate_data$RNA)
    
    tissue_2_label <- ifelse(tissue_2 == "ALL", "ALL", tissue_2)
    expression_data <- data.frame(
        Gene = rownames(aggregate_data$RNA),
        Tissue_1 = aggregate_data$RNA[, paste0(tissue_1, "_", cell_group)],
        Tissue_2 = aggregate_data$RNA[, paste0(tissue_2_label, "_", cell_group)]
    )
    
    filtered_data <- expression_data[expression_data$Tissue_1 < threshold & expression_data$Tissue_2 < threshold, ]
    
    top_genes <- filtered_data %>%
        arrange(desc(Tissue_1)) %>%
        head(top_n)
    
    p <- ggplot(filtered_data, aes(x = Tissue_1, y = Tissue_2)) +
        geom_point() +
        geom_text_repel(data = top_genes, aes(label = Gene), size = 3) +
        ggtitle(paste0("Scatter Plot of ", tissue_1, "_", cell_group, " vs ", tissue_2_label, "_", cell_group)) +
        xlab(paste0(tissue_1, "_", cell_group)) +
        ylab(paste0(tissue_2_label, "_", cell_group)) +
        theme_minimal()
    
    print(p)
}

plot_scatter_with_top_genes(sub.adi.sc, "Mature Adipocytes", "Sol", "ALL", threshold = 2e4, top_n = 10)
plot_scatter_with_top_genes(sub.adi.sc, "Mature Adipocytes", "Sol", "EDL", threshold = 2e4, top_n = 10)

## 5.4 Go analysis 

In [ ]:
library(Seurat)
library(ggplot2)
library(cowplot)
library(dplyr)
library(ggrepel)
library(clusterProfiler)
library(org.Hs.eg.db)
library(GOplot)
theme_set(theme_cowplot())

active.ident <- as.character(Idents(sub.adi.sc))
meta_data <- sub.adi.sc@meta.data
meta_data$active.ident <- active.ident

tissue_1 <- "Sol"
tissue_2 <- "EDL"
cell_group <- "Mature Adipocytes"
tissue_1_cells <- rownames(meta_data[meta_data$orig.ident == tissue_1 & meta_data$active.ident == cell_group, ])
tissue_2_cells <- rownames(meta_data[meta_data$orig.ident == tissue_2 & meta_data$active.ident == cell_group, ])

meta_data$group <- NA
meta_data$group[rownames(meta_data) %in% tissue_1_cells] <- paste0(tissue_1, "_", cell_group)
meta_data$group[rownames(meta_data) %in% tissue_2_cells] <- paste0(tissue_2, "_", cell_group)
meta_data$group <- factor(meta_data$group, levels = unique(meta_data$group))
sub.adi.sc@meta.data <- meta_data

print(table(meta_data$group))

print(head(meta_data))

Idents(sub.adi.sc) <- sub.adi.sc@meta.data$group

print(table(Idents(sub.adi.sc)))

diff_exp <- FindMarkers(sub.adi.sc, ident.1 = paste0(tissue_1, "_", cell_group), ident.2 = paste0(tissue_2, "_", cell_group))
diff_exp <- diff_exp[order(diff_exp$avg_log2FC, decreasing = TRUE), ]

min_se_threshold <- 1e-10


diff_exp_t <- FindMarkers(sub.adi.sc, ident.1 = paste0(tissue_1, "_", cell_group), ident.2 = paste0(tissue_2, "_", cell_group), test.use = "t")
diff_exp_t <- diff_exp_t[order(diff_exp_t$avg_log2FC, decreasing = TRUE), ]
head(diff_exp_t)

diff_exp_b <- FindMarkers(sub.adi.sc, ident.1 = paste0(tissue_1, "_", cell_group), ident.2 = paste0(tissue_2, "_", cell_group), test.use = "bimod")
diff_exp_b <- diff_exp_b[order(diff_exp_b$avg_log2FC, decreasing = TRUE), ]
head(diff_exp_b)


diff_exp_t$se_log2FC <- (diff_exp_t$avg_log2FC / qnorm(1 - diff_exp_t$p_val / 2))
diff_exp_t$se_log2FC[diff_exp_t$se_log2FC < min_se_threshold] <- min_se_threshold
diff_exp_t$t <- diff_exp_t$avg_log2FC / diff_exp_t$se_log2FC

diff_exp_b$se_log2FC <- (diff_exp_b$avg_log2FC / qnorm(1 - diff_exp_b$p_val / 2))
diff_exp_b$se_log2FC[diff_exp_b$se_log2FC < min_se_threshold] <- min_se_threshold
diff_exp_b$B <- diff_exp_b$avg_log2FC / diff_exp_b$se_log2FC

diff_exp <- merge(diff_exp_t, diff_exp_b[, c("avg_log2FC", "B")], by = "row.names", suffixes = c("_t", "_b"))
rownames(diff_exp) <- diff_exp$Row.names
diff_exp <- diff_exp[, -1]

colnames(diff_exp) <- gsub("\\.x", "", colnames(diff_exp))
colnames(diff_exp) <- gsub("\\.y", "_B", colnames(diff_exp))


head(diff_exp)

library(GOplot)
library(clusterProfiler)
library(org.Hs.eg.db)
library(tidyr)
library(dplyr)

filtered_genes <- diff_exp[diff_exp$p_val < 0.05 & diff_exp$avg_log2FC_b > 0.3, ]
filtered_genes$gene <- rownames(filtered_genes)

gene_list <- filtered_genes$gene

ego <- enrichGO(gene          = gene_list,
                OrgDb         = org.Hs.eg.db,
                keyType       = "SYMBOL",
                ont           = "ALL",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.05,
                qvalueCutoff  = 0.05,
                readable      = TRUE)

go_data <- data.frame(ego)
colnames(go_data)[1] = "Category"

genelist <- data.frame(ID = filtered_genes$gene, logFC = filtered_genes$avg_log2FC_b)

go_data <- go_data[, c("ID", "Description", "geneID", "p.adjust", "Category")]
colnames(go_data) <- c("ID", "Term", "Genes", "adj_pval", "Category")

go_data$Genes <- sapply(go_data$Genes, function(x) paste(unlist(strsplit(x, "/")), collapse = ", "))

required_columns <- c("Category", "ID", "Term", "Genes", "adj_pval")
go_data <- go_data[, required_columns]

go_data_long <- go_data %>%
  left_join(genelist, by = c("Genes" = "ID"))

go_data_long <- go_data_long[!is.na(go_data_long$Genes), ]

names(go_data_long)[names(go_data_long) == "logFC"] <- "logFC_gene"

go_data_long <- subset(go_data_long, select = -logFC_gene)

print(head(go_data_long))

diff_exp$ID <- rownames(diff_exp)

diff_exp$AveExpr <- (diff_exp$pct.1 + diff_exp$pct.2) / 2

colnames(diff_exp) <- c("P.Value", "logFC_t", "pct.1", "pct.2", "adj.P.Val", "se_logFC", "t", "logFC_b", "B", "ID", "AveExpr")

new_diff_exp <- diff_exp[, c("ID", "logFC_b", "AveExpr", "t", "P.Value", "adj.P.Val", "B")]

colnames(new_diff_exp) <- c("ID", "logFC", "AveExpr", "t", "P.Value", "adj.P.Val", "B")

rownames(new_diff_exp) <- 1:nrow(new_diff_exp)

circ <- circle_dat(go_data_long, new_diff_exp)
GOBubble(circ, labels = 3)

circ <- circle_dat(go_data_long, genelist)
print(head(circ))
GOBubble(circ, labels = 3)

## 5.5 GSEA 

In [ ]:
setwd("Z:\\A\\ST\\Analysis\\Results5\\gsea")
library(clusterProfiler)
library(org.Hs.eg.db)
library(DOSE)
library(ggplot2)
library(clusterProfiler)
library(enrichplot)
library(org.Hs.eg.db)
library(ggplot2)

logFC_values <- diff_exp$logFC_b
names(logFC_values) <- diff_exp$ID
logFC_values <- sort(logFC_values, decreasing = TRUE)

gsea_results <- gseGO(
  geneList = logFC_values,
  OrgDb = org.Hs.eg.db,
  keyType = "SYMBOL",
  ont = "ALL", 
  nPerm = 2000, 
  minGSSize = 5, 
  maxGSSize = 1000, 
  pvalueCutoff = 0.05,
  verbose = FALSE
)

gene_sets <- c("##")
gseaplot2(gsea_results, geneSetID = gene_sets, title = "Enrichment plot: GO:0006633", subplots = 1:3, base_size = 10)